In [ ]:
import pandas as pd
import os
import numpy as np
import string
import nltk
import re
import contractions
from imblearn.over_sampling import SVMSMOTE
# import torch
from transformers import AutoTokenizer, AutoModel

In [16]:
#getting the data
data_file_path = './data'
file_list = os.listdir(data_file_path)

swda_df = pd.concat([pd.read_csv(data_file_path+'/'+f) for f in file_list ], ignore_index=True)

display(swda_df)

print("Shape of the Data : ", swda_df.shape)

,swda_filename,ptb_basename,conversation_no,transcript_index,act_tag,caller,utterance_index,subutterance_index,text,pos,trees,ptb_treenumbers
0,sw07utt/sw_0763_3402.utt,3/sw3402,3402,0,o,A,1,1,All right. /,All/RB right/JJ ./.,NaN,NaN
1,sw07utt/sw_0763_3402.utt,3/sw3402,3402,1,sd^t,A,1,2,I guess tonight's subject is to discuss curre...,I/PRP guess/VBP tonight/RB 's/POS subject/NN i...,NaN,NaN
2,sw07utt/sw_0763_3402.utt,3/sw3402,3402,2,aa,B,2,1,Right. /,Right/UH ./.,NaN,NaN
3,sw07utt/sw_0763_3402.utt,3/sw3402,3402,3,+,A,3,1,"-- satisfied [ with, + with ] the, {F uh } --","--/: satisfied/JJ with/IN ,/, with/IN the/DT ,...",NaN,NaN
4,sw07utt/sw_0763_3402.utt,3/sw3402,3402,4,%,B,4,1,"That's, -/","That/DT 's/BES ,/,",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
223601,sw02utt/sw_0259_2020.utt,2/sw2020,2020,264,%,B,134,6,I - /,[ I/PRP ],(S-UNF (NP-SBJ (PRP I)) (-DFL- N_S)),379
223602,sw02utt/sw_0259_2020.utt,2/sw2020,2020,265,%,B,134,7,"me, - /","[ me/PRP ] ,/,","(NP (PRP me) (, ,) (-DFL- N_S))",380
223603,sw02utt/sw_0259_2020.utt,2/sw2020,2020,266,sv,B,134,8,"{C and } you never would think [ that they, + ...",and/CC [ you/PRP ] never/RB would/MD think/VB...,(S (CC and) (NP-SBJ (PRP you)) (ADVP-TMP (RB n...,381
223604,sw02utt/sw_0259_2020.utt,2/sw2020,2020,267,%,B,134,9,"{C but, } -/","but/CC ,/,","(S-UNF (CC but) (, ,) (-DFL- N_S))",382


Shape of the Data :  (223606, 12)


In [22]:
#Adding binary is_classification column based on act_tag. 

#Experiment on POS, tree and ptb_treenumbers as a feature
parameters = ['utterance_index', 'subutterance_index', 'text', 'act_tag']
swda_df = swda_df[parameters]

# Creating a new column named is_backchannel
# Marking the column as 1 if act_tag is b else 0
is_backchannel = np.where(swda_df.act_tag == 'b', 1, 0)
swda_df['is_backchannel'] = is_backchannel

display(swda_df)


,utterance_index,subutterance_index,text,act_tag,is_backchannel
0,1,1,all right,o,0
1,1,2,i guess tonight s subject is to discuss curren...,sd^t,0
2,2,1,right,aa,0
3,3,1,satisfied with with the f uh,+,0
4,4,1,that s,%,0
...,...,...,...,...,...
223601,134,6,i,%,0
223602,134,7,me,%,0
223603,134,8,c and you never would think that they that the...,sv,0
223604,134,9,c but,%,0


In [23]:
#Preprocessing the data

#converting all the text to lower
swda_df['text'] = swda_df['text'].str.lower()

#Removing Punctuations
punctuation = string.punctuation
swda_df['text'] = swda_df['text'].apply(lambda x:' '.join(word for word in x.split() if word not in punctuation))

#punkt is a sentence tokenizer which converts the text into sentences and uses unsupervised learning for abbreviations
nltk.download('punkt')

#Remove non-alphabetical character function by using regex
swda_df['text']=swda_df['text'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))
swda_df['text']=swda_df['text'].apply(lambda x: re.sub(' +', ' ', x))

#Performing word contraction using library
swda_df['no_contract'] = swda_df['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
swda_df['text'] = [' '.join(map(str, l)) for l in swda_df['no_contract']]
swda_df = swda_df.drop(['no_contract', 'act_tag'], axis='columns')

display(swda_df)

[nltk_data] Downloading package punkt to /Users/kaus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,utterance_index,subutterance_index,text,is_backchannel
0,1,1,all right,0
1,1,2,i guess tonight s subject is to discuss curren...,0
2,2,1,right,0
3,3,1,satisfied with with the f uh,0
4,4,1,that s,0
...,...,...,...,...
223601,134,6,i,0
223602,134,7,me,0
223603,134,8,c and you never would think that they that the...,0
223604,134,9,c but,0


In [45]:
#Splitting the data in X and y
X = swda_df.iloc[:, 0:3]
y = swda_df.iloc[:, 3:]

print("Features :")
display(X)

print("\n")
print("Backchanneling : ")
display(y)

Features :


,utterance_index,subutterance_index,text
0,1,1,all right
1,1,2,i guess tonight s subject is to discuss curren...
2,2,1,right
3,3,1,satisfied with with the f uh
4,4,1,that s
...,...,...,...
223601,134,6,i
223602,134,7,me
223603,134,8,c and you never would think that they that the...
223604,134,9,c but




Backchanneling : 


,is_backchannel
0,0
1,0
2,0
3,0
4,0
...,...
223601,0
223602,0
223603,0
223604,0


In [ ]:
#https://huggingface.co/docs/transformers/index
# https://www.kaggle.com/code/kausr25/bert-feature-extraction-and-fine-tuning/edit
#https://towardsdatascience.com/feature-extraction-with-bert-for-text-classification-533dde44dc2f
# https://jalammar.github.io/illustrated-bert/
import torch
from transformers import AutoTokenizer, AutoModel



In [44]:
#Balancing the class using smote
oversample = SVMSMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
 row_ix = where(y == label)[0]
 pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

ValueError: could not convert string to float: 'all right'